# Project Title
### Data Engineering Capstone Project

#### Project Summary
The project main dataset will include data on immigration to the United States, and supplementary datasets will include data on airport codes, U.S. city demographics, and temperature data.

The project follows the follow steps:
* Step 1: Scope the Project and Gather Data
* Step 2: Explore and Assess the Data
* Step 3: Define the Data Model
* Step 4: Run ETL to Model the Data
* Step 5: Complete Project Write Up

In [1]:
# Do all imports and installs here
import pandas as pd
import os
import configparser
from datetime import datetime, timedelta

from pyspark.sql import SparkSession
from pyspark.sql.functions import year, month, dayofmonth, hour, weekofyear, date_format
from pyspark.sql.functions import udf,col, array_contains, split,monotonically_increasing_id
from pyspark.sql.types import DateType, StringType
from pyspark.sql import SQLContext



In [2]:
config = configparser.ConfigParser()
config.read('keys.cfg')

os.environ['AWS_ACCESS_KEY_ID']=config['AWS_CREDENTIALS']['AWS_ACCESS_KEY_ID']
os.environ['AWS_SECRET_ACCESS_KEY']=config['AWS_CREDENTIALS']['AWS_SECRET_ACCESS_KEY']


### Step 1: Scope the Project and Gather Data

#### Scope 
Explain what you plan to do in the project in more detail. What data do you use? What is your end solution look like? What tools did you use? etc>

#### Describe and Gather Data 
Immigration Data
I94 Immigration Data comes from the U.S. National Tourism and Trade Office and contains various statistics on international visitor arrival in USA. The provided dataset contains data from April of 2016

In [3]:
# Read in the data here
path_i94 = '../../data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat'
df_i94 = pd.read_sas(path_i94, 'sas7bdat', encoding="ISO-8859-1")

In [4]:
df_i94.head(5)

,cicid,i94yr,i94mon,i94cit,i94res,i94port,arrdate,i94mode,i94addr,depdate,...,entdepu,matflag,biryear,dtaddto,gender,insnum,airline,admnum,fltno,visatype
0,6.0,2016.0,4.0,692.0,692.0,XXX,20573.0,NaN,NaN,NaN,...,U,NaN,1979.0,10282016,NaN,NaN,NaN,1.897628e+09,NaN,B2
1,7.0,2016.0,4.0,254.0,276.0,ATL,20551.0,1.0,AL,NaN,...,Y,NaN,1991.0,D/S,M,NaN,NaN,3.736796e+09,00296,F1
2,15.0,2016.0,4.0,101.0,101.0,WAS,20545.0,1.0,MI,20691.0,...,NaN,M,1961.0,09302016,M,NaN,OS,6.666432e+08,93,B2
3,16.0,2016.0,4.0,101.0,101.0,NYC,20545.0,1.0,MA,20567.0,...,NaN,M,1988.0,09302016,NaN,NaN,AA,9.246846e+10,00199,B2
4,17.0,2016.0,4.0,101.0,101.0,NYC,20545.0,1.0,MA,20567.0,...,NaN,M,2012.0,09302016,NaN,NaN,AA,9.246846e+10,00199,B2


In [5]:
df_immigration = pd.read_csv("immigration_data_sample.csv")
df_immigration.head(5)

,Unnamed: 0,cicid,i94yr,i94mon,i94cit,i94res,i94port,arrdate,i94mode,i94addr,...,entdepu,matflag,biryear,dtaddto,gender,insnum,airline,admnum,fltno,visatype
0,2027561,4084316.0,2016.0,4.0,209.0,209.0,HHW,20566.0,1.0,HI,...,NaN,M,1955.0,07202016,F,NaN,JL,5.658267e+10,00782,WT
1,2171295,4422636.0,2016.0,4.0,582.0,582.0,MCA,20567.0,1.0,TX,...,NaN,M,1990.0,10222016,M,NaN,*GA,9.436200e+10,XBLNG,B2
2,589494,1195600.0,2016.0,4.0,148.0,112.0,OGG,20551.0,1.0,FL,...,NaN,M,1940.0,07052016,M,NaN,LH,5.578047e+10,00464,WT
3,2631158,5291768.0,2016.0,4.0,297.0,297.0,LOS,20572.0,1.0,CA,...,NaN,M,1991.0,10272016,M,NaN,QR,9.478970e+10,00739,B2
4,3032257,985523.0,2016.0,4.0,111.0,111.0,CHM,20550.0,3.0,NY,...,NaN,M,1997.0,07042016,F,NaN,NaN,4.232257e+10,LAND,WT


In [6]:
# make column names inteligible
df_immigration =df_immigration.rename(columns={'cicid':'cic_id','i94yr':'year','i94mon':'month','i94cit':'cit','i94port':'port','arrdate':'arrival_date','i94mode':'mode','i94addr':'address','depdate':'departure_date','i94bir':'age','i94visa':'visa','dtadfile':'date_logged','occup':'occupation','entdepa':'arrival_flag', \
                               'entdepd':'departure_flag','entdepu':'update_flag','matflag':'match_flag','biryear':'birth_year','dtaddto':'max_stay_date','insnum':'INS_number','admnnum':'admission_number','fltno':'flight_number', 'i94res':'res'}).head()
df_immigration.columns

Index(['Unnamed: 0', 'cic_id', 'year', 'month', 'cit', 'res', 'port',
       'arrival_date', 'mode', 'address', 'departure_date', 'age', 'visa',
       'count', 'date_logged', 'visapost', 'occupation', 'arrival_flag',
       'departure_flag', 'update_flag', 'match_flag', 'birth_year',
       'max_stay_date', 'gender', 'INS_number', 'airline', 'admnum',
       'flight_number', 'visatype'],
      dtype='object')

In [7]:
# build fact table from immigration_df 

immigration_fact = df_immigration[['cic_id', 'year', 'month','arrival_date','departure_date','mode','visatype']]
immigration_fact.head()

,cic_id,year,month,arrival_date,departure_date,mode,visatype
0,4084316.0,2016.0,4.0,20566.0,20573.0,1.0,WT
1,4422636.0,2016.0,4.0,20567.0,20568.0,1.0,B2
2,1195600.0,2016.0,4.0,20551.0,20571.0,1.0,WT
3,5291768.0,2016.0,4.0,20572.0,20581.0,1.0,B2
4,985523.0,2016.0,4.0,20550.0,20553.0,3.0,WT


In [8]:
df_airport = pd.read_csv("./airport-codes_csv.csv")
df_airport.head(5)

,ident,type,name,elevation_ft,continent,iso_country,iso_region,municipality,gps_code,iata_code,local_code,coordinates
0,00A,heliport,Total Rf Heliport,11.0,NaN,US,US-PA,Bensalem,00A,NaN,00A,"-74.93360137939453, 40.07080078125"
1,00AA,small_airport,Aero B Ranch Airport,3435.0,NaN,US,US-KS,Leoti,00AA,NaN,00AA,"-101.473911, 38.704022"
2,00AK,small_airport,Lowell Field,450.0,NaN,US,US-AK,Anchor Point,00AK,NaN,00AK,"-151.695999146, 59.94919968"
3,00AL,small_airport,Epps Airpark,820.0,NaN,US,US-AL,Harvest,00AL,NaN,00AL,"-86.77030181884766, 34.86479949951172"
4,00AR,closed,Newport Hospital & Clinic Heliport,237.0,NaN,US,US-AR,Newport,NaN,NaN,NaN,"-91.254898, 35.6087"


In [9]:
df_demographics = pd.read_csv("./us-cities-demographics.csv", delimiter=";")
df_demographics.head(5)

,City,State,Median Age,Male Population,Female Population,Total Population,Number of Veterans,Foreign-born,Average Household Size,State Code,Race,Count
0,Silver Spring,Maryland,33.8,40601.0,41862.0,82463,1562.0,30908.0,2.60,MD,Hispanic or Latino,25924
1,Quincy,Massachusetts,41.0,44129.0,49500.0,93629,4147.0,32935.0,2.39,MA,White,58723
2,Hoover,Alabama,38.5,38040.0,46799.0,84839,4819.0,8229.0,2.58,AL,Asian,4759
3,Rancho Cucamonga,California,34.5,88127.0,87105.0,175232,5821.0,33878.0,3.18,CA,Black or African-American,24437
4,Newark,New Jersey,34.6,138040.0,143873.0,281913,5829.0,86253.0,2.73,NJ,White,76402


In [10]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.\
config("spark.jars.repositories", "https://repos.spark-packages.org/").\
config("spark.jars.packages", "saurfang:spark-sas7bdat:2.0.0-s_2.11").\
enableHiveSupport().getOrCreate()

df_spark = spark.read.format('com.github.saurfang.sas.spark').load('../../data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat')


In [11]:
#write to parquet
df_spark.write.mode("overwrite").parquet("sas_data")
df_spark=spark.read.parquet("sas_data")

### Step 2: Explore and Assess the Data
#### Explore the Data 
Identify data quality issues, like missing values, duplicate data, etc.


In [12]:
df_immigration.describe()

,Unnamed: 0,cic_id,year,month,cit,res,arrival_date,mode,departure_date,age,visa,count,date_logged,update_flag,birth_year,INS_number,admnum
count,5.000000e+00,5.000000e+00,5.0,5.0,5.00000,5.000000,5.000000,5.000000,5.000000,5.000000,5.0,5.0,5.000000e+00,0.0,5.000000,0.0,5.000000e+00
mean,2.090353e+06,3.195969e+06,2016.0,4.0,269.40000,262.200000,20561.200000,1.400000,20569.200000,41.400000,2.0,1.0,2.016042e+07,NaN,1974.600000,NaN,6.876748e+10
std,9.277291e+05,1.973182e+06,0.0,0.0,188.38604,194.765757,10.034939,0.894427,10.256705,25.442091,0.0,0.0,1.003494e+01,NaN,25.442091,NaN,2.423169e+10
min,5.894940e+05,9.855230e+05,2016.0,4.0,111.00000,111.000000,20550.000000,1.000000,20553.000000,19.000000,2.0,1.0,2.016041e+07,NaN,1940.000000,NaN,4.232257e+10
25%,2.027561e+06,1.195600e+06,2016.0,4.0,148.00000,112.000000,20551.000000,1.000000,20568.000000,25.000000,2.0,1.0,2.016041e+07,NaN,1955.000000,NaN,5.578047e+10
50%,2.171295e+06,4.084316e+06,2016.0,4.0,209.00000,209.000000,20566.000000,1.000000,20571.000000,26.000000,2.0,1.0,2.016042e+07,NaN,1990.000000,NaN,5.658267e+10
75%,2.631158e+06,4.422636e+06,2016.0,4.0,297.00000,297.000000,20567.000000,1.000000,20573.000000,61.000000,2.0,1.0,2.016042e+07,NaN,1991.000000,NaN,9.436200e+10
max,3.032257e+06,5.291768e+06,2016.0,4.0,582.00000,582.000000,20572.000000,3.000000,20581.000000,76.000000,2.0,1.0,2.016043e+07,NaN,1997.000000,NaN,9.478970e+10


In [13]:
df_airport.describe()

,elevation_ft
count,48069.000000
mean,1240.789677
std,1602.363459
min,-1266.000000
25%,205.000000
50%,718.000000
75%,1497.000000
max,22000.000000


In [14]:

df_demographics.describe()

,Median Age,Male Population,Female Population,Total Population,Number of Veterans,Foreign-born,Average Household Size,Count
count,2891.000000,2.888000e+03,2.888000e+03,2.891000e+03,2878.000000,2.878000e+03,2875.000000,2.891000e+03
mean,35.494881,9.732843e+04,1.017696e+05,1.989668e+05,9367.832523,4.065360e+04,2.742543,4.896377e+04
std,4.401617,2.162999e+05,2.315646e+05,4.475559e+05,13211.219924,1.557491e+05,0.433291,1.443856e+05
min,22.900000,2.928100e+04,2.734800e+04,6.321500e+04,416.000000,8.610000e+02,2.000000,9.800000e+01
25%,32.800000,3.928900e+04,4.122700e+04,8.042900e+04,3739.000000,9.224000e+03,2.430000,3.435000e+03
50%,35.300000,5.234100e+04,5.380900e+04,1.067820e+05,5397.000000,1.882200e+04,2.650000,1.378000e+04
75%,38.000000,8.664175e+04,8.960400e+04,1.752320e+05,9368.000000,3.397175e+04,2.950000,5.444700e+04
max,70.500000,4.081698e+06,4.468707e+06,8.550405e+06,156961.000000,3.212500e+06,4.980000,3.835726e+06


#### Cleaning Steps
Document steps necessary to clean the data

In [15]:
# Performing cleaning tasks here
#Get port locations from SAS text file
with open("./I94_SAS_Labels_Descriptions.SAS") as f:
    content = f.readlines()
content = [x.strip() for x in content]
ports = content[302:962]
splitted_ports = [p.split("=") for p in ports]
port_codes = [x[0].replace("'","").strip() for x in splitted_ports]
port_locations = [x[1].replace("'","").strip() for x in splitted_ports]
port_cities = [x.split(",")[0] for x in port_locations]
port_states = [x.split(",")[-1] for x in port_locations]
df_pl = pd.DataFrame({"port_code" : port_codes, "port_city": port_cities, "port_state": port_states})
df_pl.head(10)

,port_code,port_city,port_state
0,ALC,ALCAN,AK
1,ANC,ANCHORAGE,AK
2,BAR,BAKER AAF - BAKER ISLAND,AK
3,DAC,DALTONS CACHE,AK
4,PIZ,DEW STATION PT LAY DEW,AK
5,DTH,DUTCH HARBOR,AK
6,EGL,EAGLE,AK
7,FRB,FAIRBANKS,AK
8,HOM,HOMER,AK
9,HYD,HYDER,AK


In [16]:
# print first and last element in dict to check if all lines in file are covered
print(f"First port in SAS file: {df_pl['port_city'].values[0]}, last port {df_pl['port_city'].values[-1]}")
irr_ports_df = df_pl[df_pl["port_city"] == df_pl["port_state"]]
irr_ports = list(set(irr_ports_df["port_code"].values))
print(irr_ports)

First port in SAS file: ALCAN, last port No PORT Code (OSN)
['ISP', 'CPX', 'SP0', '74S', 'BKF', 'NGL', 'Y62', 'CP', 'XXX', 'FTB', 'AG', 'AMT', 'GPI', 'ASI', 'CHN', 'WTR', 'JIG', 'AUH', 'JBQ', 'BUS', 'SUS', 'MAP', 'IAG', 'XNA', 'GAC', 'DRV', 'STN', 'DEC', 'AKT', 'PHN', 'X96', 'A2A', 'ZZZ', 'JFA', 'OAI', 'MAA', 'WAS', '5T6', 'CXO', 'NC8', 'SCH', 'PFN', 'RYY', 'NK', 'YGF', '888', 'GMT', 'ADU', 'OSN', 'W55', 'JMZ', 'OLM', 'WA5', 'NYL', 'PLB', 'ATW', 'DAY', 'OGS', 'PHF', 'CLX', 'PCW', '060', 'FRG', 'MTH', 'JSJ', 'UNK', 'X44', 'FSC', 'HRL', 'OTS', '.GA', 'EGE', 'T01', 'LIT', 'TIW', 'BCM', 'VMB']


In [17]:
# drop all irregular ports from i94 
print(f"i94 data contains {len(df_i94)} rows before cleaning.")
df_i94_filtered = df_i94[~df_i94["i94port"].isin(irr_ports)]
print(f"i94 data contains {len(df_i94_filtered)} rows after removing irregular ports.")
df_i94_filtered.drop(columns=["insnum", "entdepu", "occup", "visapost"], inplace=True)
df_i94_filtered.dropna(inplace=True)
print(f"i94 data contains {len(df_i94_filtered)} rows after removing NaN values.")

i94 data contains 3096313 rows before cleaning.
i94 data contains 2995590 rows after removing irregular ports.


/opt/conda/lib/python3.6/site-packages/pandas/core/frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


i94 data contains 2306754 rows after removing NaN values.


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [18]:
# removing nan values from airport_codes
df_airport.dropna(subset=['iata_code'], inplace=True)
df_airport.head(5)

,ident,type,name,elevation_ft,continent,iso_country,iso_region,municipality,gps_code,iata_code,local_code,coordinates
223,03N,small_airport,Utirik Airport,4.0,OC,MH,MH-UTI,Utirik Island,K03N,UTK,03N,"169.852005, 11.222"
440,07FA,small_airport,Ocean Reef Club Airport,8.0,NaN,US,US-FL,Key Largo,07FA,OCA,07FA,"-80.274803161621, 25.325399398804"
594,0AK,small_airport,Pilot Station Airport,305.0,NaN,US,US-AK,Pilot Station,NaN,PQS,0AK,"-162.899994, 61.934601"
673,0CO2,small_airport,Crested Butte Airpark,8980.0,NaN,US,US-CO,Crested Butte,0CO2,CSE,0CO2,"-106.928341, 38.851918"
1088,0TE7,small_airport,LBJ Ranch Airport,1515.0,NaN,US,US-TX,Johnson City,0TE7,JCY,0TE7,"-98.62249755859999, 30.251800537100003"


In [19]:
def sas_to_date(date):
    # 1. sas dates to python datetime
    """
        datetime.strptime('2016-01-01', '%Y-%m-%d') - timedelta(20566) gives a date between the year 1959 and 1960 
        working with the date 1960-01-01 gives a more accurate date that reflects the arrival and departure dates
    """
    return (datetime.strptime('1960-01-01', '%Y-%m-%d') + timedelta(date)).strftime('%Y-%m-%d')

immigration_fact['arrival_date'] = immigration_fact.arrival_date.apply(lambda x: sas_to_date(x))
immigration_fact['departure_date'] = immigration_fact.departure_date.apply(lambda x: sas_to_date(x))

immigration_fact.head()

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.


,cic_id,year,month,arrival_date,departure_date,mode,visatype
0,4084316.0,2016.0,4.0,2016-04-22,2016-04-29,1.0,WT
1,4422636.0,2016.0,4.0,2016-04-23,2016-04-24,1.0,B2
2,1195600.0,2016.0,4.0,2016-04-07,2016-04-27,1.0,WT
3,5291768.0,2016.0,4.0,2016-04-28,2016-05-07,1.0,B2
4,985523.0,2016.0,4.0,2016-04-06,2016-04-09,3.0,WT


### Step 3: Define the Data Model
#### 3.1 Conceptual Data Model
Star schema model simple e clear structure of data and can take easier analysis and understand results.
Table	type
immigration	fact table
demographics	dimension table
airports	dimension table
#### 3.2 Mapping Out Data Pipelines
-run the ETL enter your AWS_ACCESS_KEY_ID and AWS_SECRET_ACCESS_KEY into keys.cfg file. 
-run the etl.py scrip to trigger spark job that processes and transforms the data into a combination of facts and dimension tables. 
-Check for the output in your s3 data lake to confirm successful load.

### Step 4: Run Pipelines to Model the Data 
#### 4.1 Create the data model
Build the data pipelines to create the data model.

In [20]:
def createsparksession():
    spark = SparkSession.builder\
        .config("spark.jars.repositories", "https://repos.spark-packages.org/")\
        .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:2.7.0,saurfang:spark-sas7bdat:2.0.0-s_2.11")\
        .enableHiveSupport().getOrCreate()

    return spark   

In [21]:
def read_labels(file, first_row,last_row):
    frame = {}
    frame2 = {}
    print(first_row)
    with open(file) as f:
        file_content = f.readlines()
        
        for content in file_content[first_row:last_row]:
            content = content.split("=")
            if first_row ==  303:
                code, cont= content[0].strip("\t").strip().strip("'"),content[1].strip("\t").strip().strip("''")
            else:
                code, cont = content[0].strip(),content[1].strip().strip("'")
            frame[code] =cont
    return frame

In [22]:
def process_immigration_data(spark, input_data, output_data):
    """
     Loads immigration data and processes it into a fact table
     input_data: source of data
     output_data: destination storage for processed data
    """
    # read/load immigration_data file
    immigration_data = os.path.join( input_data + "sas_data" )

    # load data
    df_immigration = spark.read.parquet(immigration_data)
    
    # extract columns to create table
    df_immigration = df_immigration.select("*")
        
    # rename columns to make them inteligible
    rename_cols =["cic_id","year","month","cit","res","port","arrival_date","mode","address","departure_date","age","visa","count","date_logged","dept_visa_issuance","occupation","arrival_flag","departure_flag","update_flag","match_flag","birth_year","max_stay_date","gender","INS_number","airline","admission_number","flight_number","visatype"]
    
    #create an iterator between collumns
    for col, rename_cols in zip(df_immigration.columns, rename_cols):
        df_immigration = df_immigration.withColumnRenamed(col, rename_cols)
    
    immigration_fact = df_immigration[
        [
            "cic_id",
            "year",
            "month",
            "arrival_date",
            "departure_date",
            "mode",
            "visatype",
        ]
    ].dropDuplicates(['cic_id'])
    
    # dimensions from immigration data
    dim_flight_details = df_immigration.select([monotonically_increasing_id().alias('id'),'cic_id','flight_number','airline']).dropDuplicates()
    dim_immigrants = df_immigration.select([monotonically_increasing_id().alias('id'),'cic_id','cit','res','visa','age','occupation','gender','address','INS_number']).dropDuplicates()
    
    # organazing data
    udf_func = udf(sas_to_date,DateType())
    immigration_fact = immigration_fact.withColumn("arrival_date",udf_func("arrival_date"))
    immigration_fact = immigration_fact.withColumn("departure_date",udf_func("departure_date"))
    file = os.path.join(input_data + "I94_SAS_Labels_Descriptions.SAS")
    countries = read_labels(file, first_row=10, last_row=298)
    cities = read_labels(file,first_row=303,last_row=962)
    countries_df = spark.createDataFrame(countries.items(), ['code', 'country']).dropDuplicates()
    cities_df = spark.createDataFrame(cities.items(), ['code', 'city']).dropDuplicates()
    cities_df = cities_df.withColumn('state', split(cities_df.city,',').getItem(1))\
            .withColumn('city', split(cities_df.city,',').getItem(0))
    cities_df = cities_df.select([monotonically_increasing_id().alias('id'),'*'])
    countries_df = countries_df.select([monotonically_increasing_id().alias('id'),'*'])
    
    # write parquet
    countries_df.write.parquet(output_data + "countries_df.parquet")
    cities_df.write.parquet(output_data + "cities_df.parquet")
    immigration_fact.write.mode('overwrite').parquet(output_data + "fact_immigration.parquet")
    dim_immigrants.write.parquet(output_data + "dim_immigrants.parquet")
    dim_flight_details.write.parquet(output_data + "dim_flight_details.parquet")

In [23]:
def process_demographics_data(spark, input_data, output_data):
    """
    loads and processes city data
    input_data: path to data source
    output_data: path to data store
    """

    # fetch data path
    city_data = os.path.join(input_data + "us-cities-demographics.csv")

    # load city data
    city_df = spark.read.load(city_data, sep=";", format="csv", header=True)

    # Demography, examine the city based on size, and structure (Gender & Ethnicity composition)
    city_demography = city_df.select(
        [
            "City",
            "State",
            "State Code",
            "Male Population",
            "Female Population",
            "Foreign-born",
            "Number of Veterans",
            "Race",
        ]
    ).dropDuplicates()
    city_demography.write.parquet(output_data + "city_demography.parquet")

    # statistics on city such as total population, and median age
    city_stats = city_df.select(
        ["City", "State Code", "Median Age", "Average Household Size", "Count"]
    ).dropDuplicates()
    city_stats.write.mode('overwrite').parquet(output_data + "city_stats.parquet")

In [24]:
def process_airport_data(spark, input_data, output_data):
    """
    loads and processes city data
    input_data: path to data source
    output_data: path to data store
    """

    # fetch data source
    airport_data = os.path.join(input_data + "airport-codes_csv.csv")
    
    # load data
    airport_df = spark.read.csv(airport_data, header=True)
    airport_df = airport_df.filter(
        (airport_df.iso_country == "US") & ~(airport_df.type == "closed")
    ).dropDuplicates()
    
    # data quality check (unique identity column)
    check_unique_id_column(airport_df,'idents')
    
    airport_dim = airport_df.select(["ident","type","name","continent","gps_code","iata_code","local_code","iso_country"]).dropDuplicates()
    airport_stats = airport_df.select(["ident", "elevation_ft", "coordinates"]).dropDuplicates()

    airport_dim.write.parquet(output_data + "airports_dim.parquet")
    airport_stats.write.parquet(output_data + "airports_stats.parquet")

#### 4.2 Data Quality Checks
Explain the data quality checks you'll perform to ensure the pipeline ran as expected. These could include:
 * Integrity constraints on the relational database (e.g., unique key, data type, etc.)
 * Unit tests for the scripts to ensure they are doing the right thing
 * Source/Count checks to ensure completeness
 
Run Quality Checks

In [27]:
cities_dtype = [('id', 'bigint'), ('code', 'string'), ('city', 'string'), ('state', 'string')]
countries_dtype = [('id', 'bigint'), ('code', 'string'), ('country', 'string')]
city_demography_dtype = [('City', 'string'), ('State', 'string'), ('State Code', 'string'), ('Male Population', 'string'), ('Female Population', 'string'), ('Foreign-born', 'string'), ('Number of Veterans', 'string'), ('Race', 'string')]
city_stats_dtype = [('City', 'string'), ('State Code', 'string'), ('Median Age', 'string'), ('Average Household Size', 'string'), ('Count', 'string')]
airports_stats_dtype = [('ident', 'string'), ('elevation_ft', 'string'), ('coordinates', 'string')]
airports_dim_dtype = [('ident', 'string'), ('type', 'string'), ('name', 'string'), ('continent', 'string'), ('gps_code', 'string'), ('iata_code', 'string'), ('local_code', 'string'), ('iso_country', 'string')]
fact_immigration_dtype = [('cic_id', 'double'), ('year', 'double'), ('month', 'double'), ('arrival_date', 'date'), ('departure_date', 'date'), ('mode', 'double'), ('visatype', 'string')]

def check_schema(spark,output_path):
    print(Path(output_path))
    output_path = Path(output_path)
    for file in output_path.iterdir():
        if file.is_dir():
            if file == 'fact_immigration.parquet' and (spark.read.parquet(str(file)).dtypes != fact_immigration_dtype):
                raise ValueError("Fact Immigration Table schema not correct")
            elif file == 'airports_dim.parquet' and (spark.read.parquet(str(file)).dtypes != airports_dim_dtype):
                raise ValueError("Airports Dim Table schema not correct")
            elif file == 'airports_stats.parquet' and (spark.read.parquet(str(file)).dtypes != airports_stats_dtype):
                raise ValueError("Aiports Stats Table schema not correct")
            elif file == 'city_stats.parquet' and (spark.read.parquet(str(file)).dtypes != city_stats_dtype):
                raise ValueError("City Stats Table schema not correct")
            elif file == 'city_demography.parquet' and (spark.read.parquet(str(file)).dtypes != city_demography_dtype):
                raise ValueError("City Demography Table schema not correct")
            elif file == 'countries.parquet' and (spark.read.parquet(str(file)).dtypes != countries_dtype):
                raise ValueError("Countries Table schema not correct")
            elif file == 'cities.parquet' and (spark.read.parquet(str(file)).dtypes != cities_dtype):
                raise ValueError("Cities Table schema not correct")
            else:
                "Table Schemas correct"
          

In [28]:
def check_empty_tables(spark,output_path):
    output_path = Path(output_path)
    for file in output_path.iterdir():
        if file.is_dir():
            df = spark.read.parquet(str(file)).count()
            if df == 0:
                print("Empty Table: ", df.split('/')[-1])
            else:
                print("Table: ", df.split('/')[-1], "is not empty")

#### 4.3 Data dictionary 
Create a data dictionary for your data model. For each field, provide a brief description of what the data is and where it came from. You can include the data dictionary in the notebook or in a separate file.

#### Step 5: Complete Project Write Up
* Clearly state the rationale for the choice of tools and technologies for the project.
* Propose how often the data should be updated and why.
* Write a description of how you would approach the problem differently under the following scenarios:
 * The data was increased by 100x.
 * The data populates a dashboard that must be updated on a daily basis by 7am every day.
 * The database needed to be accessed by 100+ people.